## Povprečne mesečne plače po občinah

### Projek pri podatkovnem rudarjenju

Podatkovno rudarjenje, projekt, `2. 3. 2020`  
**`Domen Ramšak`** **`Anže Grintal Ugovšek`**

Izbran problem (oziroma zbirka podatkov) je Povprečne mesečne plače po občinah, Slovenija, letno.

Problem zajema povprečne bruto in neto plače po občinah v Sloveniji, z dodatnimi podatki kot so opravljene (oz. plačane) nadure, število oseb, ki so opravljale nadure na mesec, število oseb, ki so prejele zaostala izplačila itd.

### Podatki



In [26]:
import pandas as pd

data = pd.read_excel('podatki/0772615.xlsx', index_col=0, header=1)  
data

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,...,2010.10,2011.10,2012.10,2013.10,2014.10,2015.10,2016.10,2017.10,2018.10,2019.10
SLOVENIJA,1157.07,1212.8,1284.79,1391.43,1438.96,1494.88,1524.65,1525.47,1523.18,1540.25,...,12.9,13.3,12.4,12.3,13.1,13.7,14.4,15.8,16.3,15.4
Ajdovščina,1061.99,1119.82,1212.73,1304.82,1327.38,1381.02,1420.68,1372.73,1369.08,1381.31,...,12.9,11.5,13.3,9,10.8,12.3,14.6,15.6,17.7,18.2
Ankaran,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,18.0,21.5,20.5,21.4,19.3
Apače,971.16,1012.75,1104.13,1231.75,1286.72,1315.45,1288.35,1256.42,1281.09,1264.54,...,8.1,6.3,8.1,8.7,8.7,7.8,10.0,9.7,10,9.2
Beltinci,920.13,989.23,1044.46,1070.86,1107.99,1136.32,1182.35,1191.19,1187.93,1214.93,...,10.6,15,15.5,12.6,12.1,9.3,9.7,11.3,11.9,13.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Železniki,1034.36,1103.56,1174.93,1163.93,1129.79,1205.48,1264.41,1245.75,1332.08,1405.7,...,7.3,6.6,5.9,7.4,9.4,10.0,13.1,17.4,16.6,12.9
Žetale,1143.91,1187.12,1255.28,1271.96,1303.34,1364.69,1358.4,1370.38,1342.08,1319.28,...,-,-,-,-,0,1.0,2.9,3.3,0.9,0.8
Žiri,995.55,1044.38,1140.53,1205.64,1130.52,1189.87,1243.5,1274.05,1299.08,1352.08,...,4.3,6.7,5.4,7.3,7.4,7.8,9.0,12.5,12.6,9.2
Žirovnica,1037.75,1137.36,1177.67,1336.57,1349.04,1382.76,1421.13,1467.57,1522.23,1471.07,...,9.7,8.4,5,9.5,8.8,7.3,9.4,13.2,12.2,11.7


In [72]:
povpMesecnaPlacaNeto = data.iloc[:, 15:30]
indeksPovpMesecnaPlacaNeto = data.iloc[:, 45:60]
povpPlacaNadure = data.iloc[:, 60:75]
povpMesecnaPlacaNetoUra = data.iloc[:, 90:105]
indeksPovpMesecnaPlacaNetoUra = data.iloc[:, 120:135]
osebeZaostalaPlacila = data.iloc[:, 135:150]
osebeNadure = data.iloc[:, 150:165]

In [74]:
count = 0
for h in range(2005, 2020):
    povpMesecnaPlacaNeto = povpMesecnaPlacaNeto.rename(columns={povpMesecnaPlacaNeto.columns[count]: h})
    indeksPovpMesecnaPlacaNeto = indeksPovpMesecnaPlacaNeto.rename(columns={indeksPovpMesecnaPlacaNeto.columns[count]: h})
    povpPlacaNadure = povpPlacaNadure.rename(columns={povpPlacaNadure.columns[count]: h})
    povpMesecnaPlacaNetoUra = povpMesecnaPlacaNetoUra.rename(columns={povpMesecnaPlacaNetoUra.columns[count]: h})
    indeksPovpMesecnaPlacaNetoUra = indeksPovpMesecnaPlacaNetoUra.rename(columns={indeksPovpMesecnaPlacaNetoUra.columns[count]: h})
    osebeZaostalaPlacila = osebeZaostalaPlacila.rename(columns={osebeZaostalaPlacila.columns[count]: h})
    osebeNadure = osebeNadure.rename(columns={osebeNadure.columns[count]: h})
    count+=1
